In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  3 10:33:00 2021

@author: jamesmason
"""
from ClassFiles.ShapeGenerator import ShapeGenerator
from ClassFiles.ChanVese import ChanVese


from PIL import Image
import datetime
import numpy as np

def create(times=1,size=(128, 128), cleansaave     ="images/clean/clean_", 
                                    dirtysave      ="images/dirty/dirty_",
                                    chansave       ="images/chan-vese/chanvese_",
                                    datacleansaave ="data/clean/clean_", 
                                    datadirtysave  ="data/dirty/dirty_",
                                    datachansave   ="data/chan-vese/chanvese_"):
    
    for i in range(times):
        e = datetime.datetime.now().strftime("%m_%d_%H_%M_%S_%f")
        #create a clean image
        shapes = ShapeGenerator(128, 128)
        shapes.add_polygon(times=3)
        shapes.add_ellipse(times=3)
        #save in clean
        shapes.image.save(fp = cleansaave+e+".png", format = 'PNG')
        np.save(file = datacleansaave+e , arr = np.array(shapes.image)/255)
        #add noise
        shapes.add_holes(40)
        shapes.add_noise()
        #save in dirty 
        shapes.image.save(fp = dirtysave+e+".png", format = 'PNG')
        np.save(file = datadirtysave+e , arr = np.array(shapes.image)/255)
        #apply chan-vese
        shapes = ChanVese(shapes.image)
        shapes.run(steps = 400,show_iterations=False)
        #save in chan-vese
        np.save(file = datachansave+e , arr = shapes.u)
        im = Image.fromarray(255*shapes.u).convert("L")
        im.save(fp =  chansave+e+".png", format = 'PNG')

In [16]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  3 10:33:00 2021

@author: jamesmason
"""
from ClassFiles.ShapeGenerator import ShapeGenerator
from ClassFiles.ChanVese import ChanVese


from PIL import Image
import datetime
import numpy as np

def create(times=1,size=(128, 128), cleansaave     ="eval/images/clean/clean_", 
                                    dirtysave      ="eval/images/dirty/dirty_",
                                    chansave       ="eval/images/chan-vese/chanvese_",
                                    datacleansaave ="eval/data/clean/clean_", 
                                    datadirtysave  ="eval/data/dirty/dirty_",
                                    datachansave   ="eval/data/chan-vese/chanvese_"):
    
    for i in range(times):
        e = datetime.datetime.now().strftime("%m_%d_%H_%M_%S_%f")
        #create a clean image
        shapes = ShapeGenerator(128, 128)
        shapes.add_polygon(times=3)
        shapes.add_ellipse(times=3)
        #save in clean
        shapes.image.save(fp = cleansaave+e+".png", format = 'PNG')
        np.save(file = datacleansaave+e , arr = np.array(shapes.image)/255)
        #add noise
        shapes.add_holes(40)
        shapes.add_noise()
        #save in dirty 
        shapes.image.save(fp = dirtysave+e+".png", format = 'PNG')
        np.save(file = datadirtysave+e , arr = np.array(shapes.image)/255)
        #apply chan-vese
        shapes = ChanVese(shapes.image)
        shapes.run(steps = 400,show_iterations=False)
        #save in chan-vese
        np.save(file = datachansave+e , arr = shapes.u)
        im = Image.fromarray(255*shapes.u).convert("L")
        im.save(fp =  chansave+e+".png", format = 'PNG')

In [17]:
create(100)

In [27]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from ClassFiles.DataLoader import get_generated_dataloader

""" for now we assume we generate all training data beforehand as numpy arrays.
we then convert to pytorch tensors and store on the cpu memory, converting to
gpu memory when needed. If memory or speed becomes an issue we could rewrite the
training data generation on the gpu using pytorch's linear algebra, and generate
it in situ while training """



def train(
    NN,
    epochs=100,
    batch_size=20,
    mu=20,
    lr=0.0001,
    device="cuda:0" if torch.cuda.is_available() else "cpu",
):
    """
    NN is the neural network, e.g. could initialise by NN = SebastianConvNet(1, 256, 256)

    groundtruth_numpy is numpy array of [batchsize, image_channels,
    image_height, image_width] groundtruth segmentations

    chanvese_numpy is numpy array of [batchsize, image_channels, image_height,
    image_width] chanvese segmentations

    The two datasets do not have to contain corresponding images for the purpose
    of training (see paper for why) INFACT THEY SHOULDN'T (should probably
    incorporate this by shuffling beforehand, or could potentially include a
    shuffle command here)

    """

    NN.to(device)
    # not sure why Sebastian doesn't use Adam, but hey
    optimiser = optim.RMSprop(NN.parameters(), lr=lr)
    
    
    groundtruth_loader = get_generated_dataloader('train', 'clean', batch_size)
    chanvese_loader = get_generated_dataloader('train', 'chan-vese', batch_size)

    for i in range(epochs):
        """
        haven't got a log keeping track of training progress at the moment
        """
        
        
        assert len(groundtruth_loader) == len(chanvese_loader)
        
        groundtruth_iter = iter(groundtruth_loader)
        chanvese_iter = iter(chanvese_loader)
        
        for i in range(len(groundtruth_loader)):
            groundtruth_batch = groundtruth_iter.next()[0]
            chanvese_batch = chanvese_iter.next()[0]
            
            assert groundtruth_batch.size() == chanvese_batch.size()

            groundtruth_batch = groundtruth_batch.to(device)
            chanvese_batch = chanvese_batch.to(device)

            batchsize = groundtruth_batch.size(0)

            # REVIEW: Unsqueezing over the 1-axis is enough for batchwise multiplication
            epsilon = torch.rand([batchsize], device=device).unsqueeze(1).unsqueeze(2).unsqueeze(3)

            intermediate_batch = (
                epsilon * groundtruth_batch + (1 - epsilon) * chanvese_batch
            )  # [batchsize, channels, height, width]
            intermediate_batch.requires_grad = True

            # apply the neural network
            groundtruth_out = NN(groundtruth_batch)  # [batchsize]
            chanvese_out = NN(chanvese_batch)  # [batchsize]
            intermediate_out = NN(intermediate_batch)  # [batchsize]

            # REVIEW: Why mean() and not sum()?
            # calculate the loss
            wasserstein_loss = (groundtruth_out - chanvese_out).mean()  # [1]

            # Set 'create_graph=True' so we can backprop a function of the
            # gradient (--> second derivatives). This is needed for implementing
            # the approximate 1-Lipschitz constraint.
            # REVIEW: Maybe we should use "retain_graph"?
            gradient = torch.autograd.grad(
                intermediate_out.sum(), intermediate_batch, create_graph=True
            )[0]
            # --> [batchsize, channels, height, width]

            gradient_loss = (
                (F.relu(gradient.square().sum((1, 2, 3)).sqrt() - 1)).square().mean()
            )  # [1]
            loss = wasserstein_loss + mu * gradient_loss  # [1]

            # backprop step
            # no need to zero the gradients of the intermediate point, since it
            # is reinitialised each batch
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
        
        print('done epoch')
    
    #torch.save(NN.state_dict(), 'shapesNN_trained')

    return NN.to("cpu")

In [23]:
from ClassFiles.networks import SebastianConvNet
NN = SebastianConvNet(1, 128, 128)

In [28]:
NN_trained = train(NN)

done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch
done epoch

In [31]:
eval_groundtruth_loader = get_generated_dataloader('eval', 'clean', batch_size = 20)
eval_chanvese_loader = get_generated_dataloader('eval', 'chan-vese', batch_size = 20)

In [40]:
eval_groundtruth_batch = iter(eval_groundtruth_loader).next()[0]
eval_chanvese_batch = iter(eval_chanvese_loader).next()[0]

In [37]:
print(NN_trained(eval_groundtruth_batch).mean().item(), NN_trained(eval_chanvese_batch).mean().item())

-45.81377410888672 -38.07417678833008


In [39]:
print(NN_trained(eval_groundtruth_batch).mean().item(), NN_trained(eval_chanvese_batch).mean().item())

-45.640769958496094 -31.28079605102539


In [41]:
print(NN_trained(eval_groundtruth_batch).mean().item(), NN_trained(eval_chanvese_batch).mean().item())

-48.58849334716797 -30.307575225830078


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from ClassFiles.DataLoader import get_generated_dataloader

""" for now we assume we generate all training data beforehand as numpy arrays.
we then convert to pytorch tensors and store on the cpu memory, converting to
gpu memory when needed. If memory or speed becomes an issue we could rewrite the
training data generation on the gpu using pytorch's linear algebra, and generate
it in situ while training """



def train(
    NN,
    epochs=100,
    batch_size=20,
    mu=20,
    lr=0.0001,
    device="cuda:0" if torch.cuda.is_available() else "cpu",
):
    """
    NN is the neural network, e.g. could initialise by NN = SebastianConvNet(1, 256, 256)

    groundtruth_numpy is numpy array of [batchsize, image_channels,
    image_height, image_width] groundtruth segmentations

    chanvese_numpy is numpy array of [batchsize, image_channels, image_height,
    image_width] chanvese segmentations

    The two datasets do not have to contain corresponding images for the purpose
    of training (see paper for why) INFACT THEY SHOULDN'T (should probably
    incorporate this by shuffling beforehand, or could potentially include a
    shuffle command here)

    """

    NN.to(device)
    # not sure why Sebastian doesn't use Adam, but hey
    optimiser = optim.RMSprop(NN.parameters(), lr=lr)
    
    
    groundtruth_loader = get_generated_dataloader('train', 'clean', batch_size)
    chanvese_loader = get_generated_dataloader('train', 'chan-vese', batch_size)
    
    eval_groundtruth_loader = get_generated_dataloader('eval', 'clean', batch_size = 100)
    eval_chanvese_loader = get_generated_dataloader('eval', 'chan-vese', batch_size = 100)
    
    
    eval_groundtruth_batch = iter(eval_groundtruth_loader).next()[0].to(device)
    eval_chanvese_batch = iter(eval_chanvese_loader).next()[0].to(device)
    with torch.no_grad():
        groundtruth_mean_value = NN(eval_groundtruth_batch).mean().item()
        chanvese_mean_value = NN(eval_chanvese_batch).mean().item()
    print('untrained performance', groundtruth_mean_value, chanvese_mean_value)
    
    for i in range(epochs):
        """
        haven't got a log keeping track of training progress at the moment
        """
        
        
        assert len(groundtruth_loader) == len(chanvese_loader)
        
        groundtruth_iter = iter(groundtruth_loader)
        chanvese_iter = iter(chanvese_loader)
        
        for i in range(len(groundtruth_loader)):
            groundtruth_batch = groundtruth_iter.next()[0]
            chanvese_batch = chanvese_iter.next()[0]
            
            assert groundtruth_batch.size() == chanvese_batch.size()

            groundtruth_batch = groundtruth_batch.to(device)
            chanvese_batch = chanvese_batch.to(device)

            batchsize = groundtruth_batch.size(0)

            # REVIEW: Unsqueezing over the 1-axis is enough for batchwise multiplication
            epsilon = torch.rand([batchsize], device=device).unsqueeze(1).unsqueeze(2).unsqueeze(3)

            intermediate_batch = (
                epsilon * groundtruth_batch + (1 - epsilon) * chanvese_batch
            )  # [batchsize, channels, height, width]
            intermediate_batch.requires_grad = True

            # apply the neural network
            groundtruth_out = NN(groundtruth_batch)  # [batchsize]
            chanvese_out = NN(chanvese_batch)  # [batchsize]
            intermediate_out = NN(intermediate_batch)  # [batchsize]

            # REVIEW: Why mean() and not sum()?
            # calculate the loss
            wasserstein_loss = (groundtruth_out - chanvese_out).mean()  # [1]

            # Set 'create_graph=True' so we can backprop a function of the
            # gradient (--> second derivatives). This is needed for implementing
            # the approximate 1-Lipschitz constraint.
            # REVIEW: Maybe we should use "retain_graph"?
            gradient = torch.autograd.grad(
                intermediate_out.sum(), intermediate_batch, create_graph=True
            )[0]
            # --> [batchsize, channels, height, width]

            gradient_loss = (
                (F.relu(gradient.square().sum((1, 2, 3)).sqrt() - 1)).square().mean()
            )  # [1]
            loss = wasserstein_loss + mu * gradient_loss  # [1]

            # backprop step
            # no need to zero the gradients of the intermediate point, since it
            # is reinitialised each batch
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
        
        eval_groundtruth_batch = iter(eval_groundtruth_loader).next()[0].to(device)
        eval_chanvese_batch = iter(eval_chanvese_loader).next()[0].to(device)
        with torch.no_grad():
            groundtruth_mean_value = NN(eval_groundtruth_batch).mean().item()
            chanvese_mean_value = NN(eval_chanvese_batch).mean().item()
        print('done epoch', groundtruth_mean_value, chanvese_mean_value)
    
    #torch.save(NN.state_dict(), 'shapesNN_trained2')

    return NN.to("cpu")

In [2]:
from ClassFiles.networks import SebastianConvNet
NN = SebastianConvNet(1, 128, 128)
NN_trained2 = train(NN)

untrained performance 0.038864150643348694 0.038880396634340286
done epoch 12.618231773376465 18.813182830810547
done epoch 63.58605194091797 71.1834487915039
done epoch 15.920390129089355 22.874990463256836
done epoch 49.21866226196289 56.95798110961914
done epoch 45.18325424194336 53.36827087402344
done epoch 16.57179832458496 24.142093658447266
done epoch 4.840348243713379 12.920366287231445
done epoch -12.929286003112793 -4.847332000732422
done epoch 7.324130535125732 16.498716354370117
done epoch 12.941987991333008 22.389266967773438
done epoch 2.5564610958099365 9.952454566955566
done epoch 22.267440795898438 31.339984893798828
done epoch 16.84214973449707 26.06429672241211
done epoch 20.55097770690918 29.98940086364746
done epoch 21.843690872192383 31.192262649536133
done epoch 12.76811695098877 18.48917579650879
done epoch 13.273622512817383 22.387664794921875
done epoch -0.3863922655582428 8.586007118225098
done epoch 19.88096809387207 30.033798217773438
done epoch 3.320398569

In [3]:
import numpy as np
import torch
import torch.nn.functional as F

"""
the regularisation parameter lambda CANNOT be initialised in the same way for segmentation as for denoising
"""
"""
the function unreg_mini in Sebastian's paper shouldn't need to be redone here, as chanvese should already be a good starting point for reconstruction
"""


"""
must recalculate data fitting term in pytorch, so we can compute gradients
"""
"""
I THINK THIS IS THE IMPLEMENTATION MIKE WANTED
"""


def data_fitting(
    chanvese_batch,
    noisy_batch,
    lambda_chanvese=1,
    threshold=0.5,
    c1=None,
    c2=None,
    alpha=None,
):
    """
    chanvese_batch & noisy_batch must a torch.tensor (ideally on gpu) of size [batchsize, 1, height, width]
    noisy_batch contains the corresponding noisy images to chanvese_batch
    threshold is used to find the segmentation boundary (for the purpose of calculating c1, c2) from chanvese_batch
    """
    assert chanvese_batch.size() == noisy_batch.size()
    assert chanvese_batch.size(1) == 1  # require greyscale image, i.e. only one channel

    # calculate c1, c2 implicity from u
    """
    WE DO NOT want to backprop along c1, c2 when performing the reconstruction (algorithm 2), only relevant when c1, c2 are calculated implicitly
    Actually not a problem, since no operation below is going to preserve requires_grad = true
    """
    if c1 == None:
        c1 = (noisy_batch * (chanvese_batch > threshold)).mean((1, 2, 3))  # [batchsize]
    if c2 == None:
        c2 = (noisy_batch * (chanvese_batch <= threshold)).mean(
            (1, 2, 3)
        )  # [batchsize]

    chanvese_term = lambda_chanvese * (
        (noisy_batch - c1.unsqueeze(1).unsqueeze(2).unsqueeze(3)).square()
        - (noisy_batch - c2.unsqueeze(1).unsqueeze(2).unsqueeze(3)).square()
    )  # [batchsize, 1, height, width]

    # calculate alpha implicity from u, lambda, c1, and c2?
    """
    WE DO NOT want to backprop along alpha when performing the reconstruction (algorithm 2), only relevant when alpha is calculated implicitly
    hence .detach() below
    """
    if alpha == None:
        alpha = (
            chanvese_term.detach().abs().max(3)[0].max(2)[0].max(1)[0]
        )  # [batchsize]

    penality_term = 2 * (
        (chanvese_batch - 0.5).abs() - 1
    )  # [batchsize, 1, height, width]
    penality_term = penality_term * (penality_term > 0)  # [batchsize, 1, height, width]

    """
    integral over domain is just done by taking the mean, should just correspond to scaling lambda_reg accordingly in reconstruct (below)
    """
    datafitting_term = (
        chanvese_term * chanvese_batch
        + alpha.unsqueeze(1).unsqueeze(2).unsqueeze(3) * penality_term
    ).mean(
        (1, 2, 3)
    )  # [batchsize]

    return datafitting_term  # [batchsize]


"""
ALGORITHM 2:
simultaneously perform a number of gradient descent steps on a full batch of chanvese segmentations, or already partialy reconstructed segmentations
(both would take the argument chanvese_batch below)
noisy_batch contains the corresponding noisy images to chanvese_batch (for the purpose of the datafitting term above)
"""


def reconstruct(
    chanvese_batch, noisy_batch, NN, lambda_reg, epsilon, reconstruction_steps=1
):
    """
    chanvese_batch & noisy_batch must be a torch.tensor of size [batchsize, channels, height, width]
    NN is the learnt regulariser
    lambda_reg is how much we weight the regularising term (not the datafitting term) when reconstructing the solution according to algorithm 2
    """
    device = next(NN.parameters()).device  # trick to find device NN is stored on
    reconstructed_batch = chanvese_batch.to(
        device
    ).detach()  # transfer chanvese_batch to same device NN is stored on, detach just incase
    noisy_batch_copy = noisy_batch.to(
        device
    )  # transfer noisy_batch to same device NN is stored on

    for i in range(reconstruction_steps):
        reconstructed_batch.requires_grad = True  # set requires_grad to True, gradients are initialised at zero, and entire backprop graph will be recreated (not the most efficient way, as autograd graph has to be recreated each time)

        """
        data_fitting function not yet implemented
        """
        datafitting = data_fitting(reconstructed_batch, noisy_batch_copy)  # [batchsize]
        regularising = NN(reconstructed_batch)  # [batchsize]

        error = datafitting + lambda_reg * regularising  # [batchsize]
        error = error.sum()  # [1], trick to compute all gradients in one go

        gradients = torch.autograd.grad(error, reconstructed_batch)[0]
        reconstructed_batch = (
            reconstructed_batch - epsilon * gradients
        ).detach()  # detaching from previous autograd which also sets requires_grad to False

    return (
        reconstructed_batch.to(chanvese_batch.device),
        noisy_batch,
    )  # send back to original device


"""
a quick function for evaluating the quality of the reconstructed segmentation according to the L2 difference between it and groundtruth
"""


def quality(reconstructed_batch, groundtruth_batch):
    """
    reconstructed_batch, and groundtruth_batch must be torch.tensors of the same size [batchsize, channels, height, width]
    """
    return (
        (reconstructed_batch - groundtruth_batch).square().sum((1, 2, 3)).sqrt()
    )  # [batchsize]


"""
analogue of Sebastian's function log_minimum (except without storing any data in logs), which keeps reconstructing solutions until their quality (as defined above, i.e. requiring knowledge of the ground truth) no longer keeps decreasing

idea is to use this to evaluate performance of NN
"""


def minimum(chanvese_batch, noisy_batch, groundtruth_batch, NN, lambda_reg, epsilon):

    assert chanvese_batch.size() == noisy_batch.size()
    assert chanvese_batch.size() == groundtruth_batch.size()
    assert chanvese_batch.device == noisy_batch.device
    assert chanvese_batch.device == groundtruth_batch.device
    batchsize = chanvese_batch.size(0)
    device = chanvese_batch.device

    todo_mask = torch.ones([batchsize], dtype=torch.bool, device=device)
    chanvese_todo = chanvese_batch
    noisy_todo = noisy_batch
    groundtruth_todo = groundtruth_batch
    quality_prev = torch.full([batchsize], float("inf"), device=device)
    minimum_batch = torch.empty_like(chanvese_todo)
    final_quality = torch.empty_like(quality_prev)
    steps = torch.zeros_like(quality_prev)

    while todo_mask.sum():
        print(todo_mask.sum())
        steps += todo_mask

        chanvese_todo = reconstruct(chanvese_todo, noisy_todo, NN, lambda_reg, epsilon)[0]
        quality_new = quality(chanvese_todo, groundtruth_todo)
        done_mask = quality_new > quality_prev

        done_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(
            todo_mask, done_mask
        )

        minimum_batch.masked_scatter_(
            done_mask_unravel.unsqueeze(1)
            .unsqueeze(2)
            .unsqueeze(3)
            .expand(minimum_batch.size()),
            chanvese_todo.masked_select(
                done_mask.unsqueeze(1)
                .unsqueeze(2)
                .unsqueeze(3)
                .expand(chanvese_todo.size())
            ),
        )

        final_quality.masked_scatter_(
            done_mask_unravel, quality_new.masked_select(done_mask)
        )
        todo_mask.masked_fill_(done_mask_unravel, False)

        chanvese_todo = chanvese_todo.masked_select(
            done_mask.logical_not()
            .unsqueeze(1)
            .unsqueeze(2)
            .unsqueeze(3)
            .expand(chanvese_todo.size())
        ).view(
            -1, chanvese_batch.size(1), chanvese_batch.size(2), chanvese_batch.size(3)
        )
        noisy_todo = noisy_todo.masked_select(
            done_mask.logical_not()
            .unsqueeze(1)
            .unsqueeze(2)
            .unsqueeze(3)
            .expand(noisy_todo.size())
        ).view(
            -1, chanvese_batch.size(1), chanvese_batch.size(2), chanvese_batch.size(3)
        )
        groundtruth_todo = groundtruth_todo.masked_select(
            done_mask.logical_not()
            .unsqueeze(1)
            .unsqueeze(2)
            .unsqueeze(3)
            .expand(groundtruth_todo.size())
        ).view(
            -1, chanvese_batch.size(1), chanvese_batch.size(2), chanvese_batch.size(3)
        )
        quality_prev = quality_new.masked_select(done_mask.logical_not())

    return (
        minimum_batch,
        final_quality,
        steps,
    )  # outputs the final (optimal) reconstruction, their corresponding quality, and the reconstruction steps required


In [37]:
eval_groundtruth_loader = get_generated_dataloader('eval', 'clean', batch_size = 100, shuffle = False)
eval_chanvese_loader = get_generated_dataloader('eval', 'chan-vese', batch_size = 100, shuffle = False)
eval_noisy_loader = get_generated_dataloader('eval', 'dirty', batch_size = 100, shuffle = False)

eval_groundtruth_batch = iter(eval_groundtruth_loader).next()[0]
eval_chanvese_batch = iter(eval_chanvese_loader).next()[0]
eval_noisy_batch = iter(eval_noisy_loader).next()[0]

In [92]:
minimum_batch, _ = reconstruct(eval_chanvese_batch, eval_noisy_batch, NN_trained2, 1, 0.001, 50)

In [93]:
quality(minimum_batch, eval_groundtruth_batch)

tensor([20.7308, 22.2587, 21.2586, 20.2870, 21.7970, 25.1236, 21.2068, 18.8730,
        26.8303, 22.2677, 31.8682, 21.3523, 24.3287, 18.9921, 23.1867, 17.1488,
        21.1736, 20.3297, 20.7431, 20.4539, 25.2307, 21.0587, 23.1987, 24.5889,
        23.3309, 17.5778, 24.2103, 20.2828, 18.7939, 20.5546, 24.8759, 26.5289,
        17.9498, 21.6536, 19.8809, 19.0416, 18.6636, 29.5160, 19.5933, 22.6014,
        18.9797, 22.4489, 21.6593, 18.8388, 23.4502, 23.9608, 18.1446, 22.6261,
        24.9773, 17.1516, 23.6668, 20.2068, 22.4793, 17.1010, 20.3928, 22.2331,
        17.3984, 27.0341, 18.4000, 28.0716, 25.2354, 23.7372, 25.0803, 25.4750,
        20.9388, 21.2452, 26.2204, 20.4299, 19.3649, 23.1272, 18.5059, 22.4491,
        23.8683, 25.7508, 23.3741, 19.3996, 20.1539, 20.5131, 18.9185, 28.9504,
        18.9385, 24.8252, 18.1016, 26.8381, 24.4966, 26.6884, 19.7876, 20.4098,
        23.4061, 19.1060, 21.4435, 21.5396, 20.2280, 18.2687, 18.0217, 16.4721,
        20.0200, 19.8940, 19.0621, 24.32

In [94]:
quality(eval_chanvese_batch, eval_groundtruth_batch)

tensor([19.2556, 21.5176, 19.8189, 18.7138, 21.1041, 23.9811, 19.9073, 16.9417,
        26.8937, 20.6464, 31.6227, 20.3902, 23.7194, 18.3315, 22.3634, 15.5090,
        19.6037, 18.9297, 18.8746, 18.9002, 24.0845, 19.4287, 22.6050, 23.9401,
        22.2961, 15.8771, 23.4152, 18.4406, 17.6938, 19.0591, 24.0654, 25.6187,
        16.4093, 20.3504, 19.0126, 17.4013, 17.1774, 28.7044, 18.3316, 21.9211,
        17.2012, 21.3382, 20.4999, 17.7858, 22.0503, 23.6145, 16.5793, 21.5948,
        24.0078, 15.7314, 21.9132, 18.7845, 21.2659, 15.0023, 19.0546, 20.8455,
        16.1690, 26.3345, 16.9159, 27.3111, 24.2817, 23.1646, 23.9969, 24.7396,
        19.6489, 20.1087, 25.3276, 19.0618, 18.0218, 22.2856, 17.0062, 20.7066,
        22.5429, 24.8365, 22.1405, 18.0067, 18.8572, 18.9382, 17.6190, 28.4396,
        16.6455, 23.8683, 16.7484, 26.1403, 23.6404, 26.1381, 18.3444, 18.5999,
        21.9343, 17.3227, 20.0019, 20.6936, 19.0761, 17.1766, 16.4064, 15.3241,
        18.4786, 18.6037, 17.7078, 23.43

In [95]:
print(NN_trained2(minimum_batch).mean().item(), NN_trained2(eval_groundtruth_batch).mean().item(), NN_trained2(eval_chanvese_batch).mean().item()) 

-6.556939601898193 -5.763604640960693 5.858641147613525


In [96]:
minimum_batch_numpy = minimum_batch.numpy()
minimum_batch_numpy_squeeze = minimum_batch_numpy.squeeze()

In [97]:
from PIL import Image
for i in range(100):
    im = Image.fromarray(255*minimum_batch_numpy_squeeze[i]).convert("L")
    im.save(fp =  "eval/images/reconstructed/reconstructed_"+str(i).zfill(2)+".png", format = 'PNG')